In [5]:
import torch

In [6]:
import torch.nn as nn

In [ ]:
class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads)
        # heads = how many parts we split it into
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim - embed_size // heads # TODO: what is this double slash
        
        assert (self.head_dim * heads = embed_size), "Embed size needs to be divisibele by heads"
        
        self.values = nn.Linear(self.head_dim, self.head_tim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_tim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_fim, bias=False)
        self.fc_out = nn.Linear(head*self.head_dim, embed_size)
        